In [1]:
import argparse

In [2]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import Optimizer, nihgcn
from myutils import *
from sampler import RandomSampler
from sklearn.model_selection import KFold

In [3]:
class Args:
    def __init__(self):
        self.device = "cuda:0"  # cuda:number or cpu
        self.data = "nci"  # Dataset{gdsc or ccle}
        self.lr = 0.001  # the learning rate
        self.wd = 1e-5  # the weight decay for l2 normalizaton
        self.layer_size = [1024, 1024]  # Output sizes of every layer
        self.alpha = 0.25  # the scale for balance gcn and ni
        self.gamma = 8  # the scale for sigmod
        self.epochs = 1000  # the epochs for model


args = Args()

In [4]:
res, drug_finger, exprs, null_mask, pos_num = load_data(args)

load nci


In [9]:
k = 5
kfold = KFold(n_splits=k, shuffle=True, random_state=42)

true_datas = pd.DataFrame()
predict_datas = pd.DataFrame()

for train_index, test_index in kfold.split(np.arange(pos_num)):
    sampler = RandomSampler(res, train_index, test_index, null_mask)
    model = nihgcn(
        adj_mat=sampler.train_data,
        cell_exprs=exprs,
        drug_finger=drug_finger,
        layer_size=args.layer_size,
        alpha=args.alpha,
        gamma=args.gamma,
        device=args.device,
    ).to(args.device)
    opt = Optimizer(
        model,
        sampler.train_data,
        sampler.test_data,
        sampler.test_mask,
        sampler.train_mask,
        roc_auc,
        lr=args.lr,
        wd=args.wd,
        epochs=args.epochs,
        device=args.device,
    ).to(args.device)
    true_data, predict_data = opt()
    true_datas = pd.concat([true_datas, translate_result(true_data)], ignore_index=True)
    predict_datas = pd.concat(
        [predict_datas, translate_result(predict_data)], ignore_index=True
    )

epoch:   0 loss:0.696969 auc:0.5063
epoch:  20 loss:0.345005 auc:0.8474
epoch:  40 loss:0.287488 auc:0.8563
epoch:  60 loss:0.259091 auc:0.8559
epoch:  80 loss:0.242383 auc:0.8544
epoch: 100 loss:0.232310 auc:0.8522
epoch: 120 loss:0.226576 auc:0.8500
epoch: 140 loss:0.223378 auc:0.8482
epoch: 160 loss:0.221643 auc:0.8467
epoch: 180 loss:0.220440 auc:0.8460
epoch: 200 loss:0.219676 auc:0.8452
epoch: 220 loss:0.219135 auc:0.8444
epoch: 240 loss:0.219045 auc:0.8439
epoch: 260 loss:0.218726 auc:0.8441
epoch: 280 loss:0.218309 auc:0.8430
epoch: 300 loss:0.218597 auc:0.8420
epoch: 320 loss:0.217924 auc:0.8432
epoch: 340 loss:0.218327 auc:0.8432
epoch: 360 loss:0.217767 auc:0.8430
epoch: 380 loss:0.217208 auc:0.8427
epoch: 400 loss:0.216916 auc:0.8421
epoch: 420 loss:0.217030 auc:0.8421
epoch: 440 loss:0.217771 auc:0.8424
epoch: 460 loss:0.216698 auc:0.8409
epoch: 480 loss:0.216338 auc:0.8403
epoch: 500 loss:0.217224 auc:0.8397
epoch: 520 loss:0.215822 auc:0.8401
epoch: 540 loss:0.216794 auc

In [13]:
true_datas.to_csv("true_nci.csv")
predict_datas.to_csv("pred_nci.csv")